In [11]:
# SEMANA 3: PREPARACIÓN DE DATOS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)

print("FASE 3: PREPARACIÓN DE DATOS")
print("="*60)

FASE 3: PREPARACIÓN DE DATOS


In [12]:
# Cargar datos validados
covid_df = pd.read_parquet('../data/02_intermediate/covid_compact_validated.parquet')
vacc_df = pd.read_parquet('../data/02_intermediate/vaccination_global_validated.parquet')

print(f"COVID: {covid_df.shape}")
print(f"Vacunación: {vacc_df.shape}")

COVID: (521908, 61)
Vacunación: (203057, 29)


In [13]:
def detailed_missing_analysis(df, name):
    """Análisis detallado para estrategia de imputación"""
    
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    
    analysis = pd.DataFrame({
        'column': missing.index,
        'missing_count': missing.values,
        'missing_pct': missing_pct.values,
        'dtype': df.dtypes.values
    })
    
    analysis = analysis[analysis['missing_count'] > 0].sort_values('missing_pct', ascending=False)
    
    # Categorizar columnas por % de missing
    analysis['strategy'] = analysis['missing_pct'].apply(lambda x:
        'DROP' if x > 70 else
        'IMPUTE_COMPLEX' if x > 30 else
        'IMPUTE_SIMPLE' if x > 0 else
        'KEEP'
    )
    
    print(f"\n{'='*70}")
    print(f"ANÁLISIS MISSING - {name}")
    print(f"{'='*70}")
    
    # Resumen por estrategia
    print("\nESTRATEGIA DE LIMPIEZA:")
    for strategy in ['DROP', 'IMPUTE_COMPLEX', 'IMPUTE_SIMPLE']:
        cols = analysis[analysis['strategy'] == strategy]
        print(f"\n{strategy}:")
        print(f"  Columnas: {len(cols)}")
        if len(cols) > 0:
            print(f"  Range: {cols['missing_pct'].min():.1f}% - {cols['missing_pct'].max():.1f}%")
            print(f"  Ejemplos: {cols['column'].head(5).tolist()}")
    
    return analysis

covid_missing_analysis = detailed_missing_analysis(covid_df, "COVID-19")
vacc_missing_analysis = detailed_missing_analysis(vacc_df, "VACUNACIÓN")


ANÁLISIS MISSING - COVID-19

ESTRATEGIA DE LIMPIEZA:

DROP:
  Columnas: 30
  Range: 80.1% - 100.0%
  Ejemplos: ['human_development_index', 'weekly_icu_admissions_per_million', 'weekly_icu_admissions', 'excess_mortality_cumulative_per_million', 'excess_mortality_cumulative_absolute']

IMPUTE_COMPLEX:
  Columnas: 9
  Range: 35.9% - 64.4%
  Ejemplos: ['reproduction_rate', 'stringency_index', 'new_vaccinations_smoothed_per_million', 'new_vaccinations_smoothed', 'new_people_vaccinated_smoothed']

IMPUTE_SIMPLE:
  Columnas: 20
  Range: 2.7% - 21.9%
  Ejemplos: ['gdp_per_capita', 'diabetes_prevalence', 'continent', 'population_density', 'median_age']

ANÁLISIS MISSING - VACUNACIÓN

ESTRATEGIA DE LIMPIEZA:

DROP:
  Columnas: 4
  Range: 71.0% - 99.3%
  Ejemplos: ['share_of_boosters', 'total_vaccinations_no_boosters_per_hundred_interpolated', 'total_boosters', 'total_boosters_per_hundred']

IMPUTE_COMPLEX:
  Columnas: 8
  Range: 59.0% - 66.1%
  Ejemplos: ['daily_vaccinations', 'people_fully_vac

In [14]:
# Columnas a eliminar (>70% missing)
covid_drop_cols = covid_missing_analysis[
    covid_missing_analysis['strategy'] == 'DROP'
]['column'].tolist()

vacc_drop_cols = vacc_missing_analysis[
    vacc_missing_analysis['strategy'] == 'DROP'
]['column'].tolist()

print("\nCOLUMNAS A ELIMINAR:")
print(f"\nCOVID ({len(covid_drop_cols)} columnas):")
for col in covid_drop_cols[:10]:  # Primeras 10
    pct = covid_missing_analysis[covid_missing_analysis['column']==col]['missing_pct'].values[0]
    print(f"  - {col}: {pct:.1f}% missing")

print(f"\nVACUNACIÓN ({len(vacc_drop_cols)} columnas):")
for col in vacc_drop_cols:
    pct = vacc_missing_analysis[vacc_missing_analysis['column']==col]['missing_pct'].values[0]
    print(f"  - {col}: {pct:.1f}% missing")


COLUMNAS A ELIMINAR:

COVID (30 columnas):
  - human_development_index: 100.0% missing
  - weekly_icu_admissions_per_million: 97.9% missing
  - weekly_icu_admissions: 97.9% missing
  - excess_mortality_cumulative_per_million: 97.4% missing
  - excess_mortality_cumulative_absolute: 97.4% missing
  - excess_mortality_cumulative: 97.4% missing
  - excess_mortality: 97.4% missing
  - weekly_hosp_admissions_per_million: 95.3% missing
  - weekly_hosp_admissions: 95.3% missing
  - icu_patients_per_million: 92.5% missing

VACUNACIÓN (4 columnas):
  - share_of_boosters: 99.3% missing
  - total_vaccinations_no_boosters_per_hundred_interpolated: 71.3% missing
  - total_boosters: 71.0% missing
  - total_boosters_per_hundred: 71.0% missing


In [15]:
# Columnas críticas a mantener (aunque tengan missing)
covid_keep_cols = [
    'country', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths',
    'total_cases_per_million', 'total_deaths_per_million',
    'population', 'population_density', 'median_age', 'gdp_per_capita',
    'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index'
]

vacc_keep_cols = [
    'country', 'date', 'total_vaccinations', 'people_vaccinated', 
    'people_fully_vaccinated', 'total_vaccinations_per_hundred',
    'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred'
]

# Verificar cuáles existen
covid_available = [col for col in covid_keep_cols if col in covid_df.columns]
vacc_available = [col for col in vacc_keep_cols if col in vacc_df.columns]

print("COLUMNAS SELECCIONADAS:")
print(f"\nCOVID ({len(covid_available)}):")
print(covid_available)
print(f"\nVACUNACIÓN ({len(vacc_available)}):")
print(vacc_available)

# Crear datasets reducidos
covid_clean = covid_df[covid_available].copy()
vacc_clean = vacc_df[vacc_available].copy()

print(f"\nDATASETS REDUCIDOS:")
print(f"COVID: {covid_df.shape} → {covid_clean.shape}")
print(f"Vacunación: {vacc_df.shape} → {vacc_clean.shape}")

COLUMNAS SELECCIONADAS:

COVID (15):
['country', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'total_deaths_per_million', 'population', 'population_density', 'median_age', 'gdp_per_capita', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index']

VACUNACIÓN (8):
['country', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred']

DATASETS REDUCIDOS:
COVID: (521908, 61) → (521908, 15)
Vacunación: (203057, 29) → (203057, 8)


In [16]:
def impute_missing_values(df, strategy_dict):
    """Imputa valores según estrategias definidas"""
    df_imputed = df.copy()
    
    for col, strategy in strategy_dict.items():
        if col not in df_imputed.columns:
            continue
            
        missing_count = df_imputed[col].isnull().sum()
        if missing_count == 0:
            continue
        
        if strategy == 'median':
            df_imputed[col].fillna(df_imputed[col].median(), inplace=True)
        elif strategy == 'mean':
            df_imputed[col].fillna(df_imputed[col].mean(), inplace=True)
        elif strategy == 'forward_fill':
            df_imputed[col].fillna(method='ffill', inplace=True)
        elif strategy == 'zero':
            df_imputed[col].fillna(0, inplace=True)
        
        print(f"  {col}: {missing_count} valores imputados ({strategy})")
    
    return df_imputed

# Estrategias para COVID
covid_strategies = {
    'total_cases': 'zero',
    'new_cases': 'zero',
    'total_deaths': 'zero',
    'new_deaths': 'zero',
    'total_cases_per_million': 'zero',
    'total_deaths_per_million': 'zero',
    'population_density': 'median',
    'median_age': 'median',
    'gdp_per_capita': 'median',
    'hospital_beds_per_thousand': 'median',
    'life_expectancy': 'median',
    'human_development_index': 'median'
}

# Estrategias para Vacunación
vacc_strategies = {
    'total_vaccinations': 'zero',
    'people_vaccinated': 'zero',
    'people_fully_vaccinated': 'zero',
    'total_vaccinations_per_hundred': 'zero',
    'people_vaccinated_per_hundred': 'zero',
    'people_fully_vaccinated_per_hundred': 'zero'
}

print("\nIMPUTANDO VALORES:")
print("\nCOVID:")
covid_imputed = impute_missing_values(covid_clean, covid_strategies)

print("\nVACUNACIÓN:")
vacc_imputed = impute_missing_values(vacc_clean, vacc_strategies)

# Verificar que no quedan missing
print(f"\nMISSING RESTANTES:")
print(f"COVID: {covid_imputed.isnull().sum().sum()}")
print(f"Vacunación: {vacc_imputed.isnull().sum().sum()}")


IMPUTANDO VALORES:

COVID:
  total_cases: 14197 valores imputados (zero)
  new_cases: 17339 valores imputados (zero)
  total_deaths: 14197 valores imputados (zero)
  new_deaths: 16503 valores imputados (zero)
  total_cases_per_million: 14197 valores imputados (zero)
  total_deaths_per_million: 14197 valores imputados (zero)
  population_density: 25087 valores imputados (median)
  median_age: 23040 valores imputados (median)
  gdp_per_capita: 114356 valores imputados (median)
  hospital_beds_per_thousand: 206260 valores imputados (median)
  life_expectancy: 21001 valores imputados (median)
  human_development_index: 521908 valores imputados (median)

VACUNACIÓN:
  total_vaccinations: 119835 valores imputados (zero)
  people_vaccinated: 124046 valores imputados (zero)
  people_fully_vaccinated: 126046 valores imputados (zero)
  total_vaccinations_per_hundred: 119835 valores imputados (zero)
  people_vaccinated_per_hundred: 124046 valores imputados (zero)
  people_fully_vaccinated_per_hu

In [17]:
print("\nCREANDO FEATURES...")

# Features para COVID
covid_features = covid_imputed.copy()

# 1. Velocidad de crecimiento de casos
covid_features['cases_growth_rate'] = covid_features.groupby('country')['total_cases'].pct_change()

# 2. Tasa de mortalidad
covid_features['death_rate'] = (
    covid_features['total_deaths'] / covid_features['total_cases'] * 100
).replace([np.inf, -np.inf], 0).fillna(0)

# 3. Índice de capacidad sanitaria
covid_features['healthcare_capacity_index'] = (
    covid_features['hospital_beds_per_thousand'] * 
    covid_features['gdp_per_capita'] / 10000
).fillna(0)

# Features para Vacunación
vacc_features = vacc_imputed.copy()

# 1. Velocidad de vacunación
vacc_features['vaccination_speed'] = vacc_features.groupby('country')[
    'people_vaccinated_per_hundred'
].diff()

# 2. Eficiencia de vacunación (totalmente vacunados vs parcialmente)
vacc_features['vaccination_efficiency'] = (
    vacc_features['people_fully_vaccinated'] / 
    vacc_features['people_vaccinated']
).replace([np.inf, -np.inf], 0).fillna(0) * 100

print(f"\nFEATURES CREADOS:")
print(f"COVID: {len(covid_features.columns) - len(covid_imputed.columns)} nuevos")
print(f"Vacunación: {len(vacc_features.columns) - len(vacc_imputed.columns)} nuevos")

print("\nNuevas columnas COVID:")
print([col for col in covid_features.columns if col not in covid_imputed.columns])
print("\nNuevas columnas Vacunación:")
print([col for col in vacc_features.columns if col not in vacc_imputed.columns])


CREANDO FEATURES...

FEATURES CREADOS:
COVID: 3 nuevos
Vacunación: 2 nuevos

Nuevas columnas COVID:
['cases_growth_rate', 'death_rate', 'healthcare_capacity_index']

Nuevas columnas Vacunación:
['vaccination_speed', 'vaccination_efficiency']


In [18]:
# Preparar para merge
print("\nINTEGRANDO DATASETS...")

# Último registro por país para datos estáticos
covid_latest = covid_features.sort_values('date').groupby('country').last().reset_index()
vacc_latest = vacc_features.sort_values('date').groupby('country').last().reset_index()

# Merge
integrated_df = covid_latest.merge(
    vacc_latest,
    on='country',
    how='left',
    suffixes=('_covid', '_vacc')
)

print(f"\nDATASET INTEGRADO:")
print(f"Filas: {len(integrated_df)}")
print(f"Columnas: {len(integrated_df.columns)}")
print(f"Países: {integrated_df['country'].nunique()}")

# Vista previa
print("\nColumnas finales:")
print(integrated_df.columns.tolist())


INTEGRANDO DATASETS...

DATASET INTEGRADO:
Filas: 262
Columnas: 27
Países: 262

Columnas finales:
['country', 'date_covid', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'total_deaths_per_million', 'population', 'population_density', 'median_age', 'gdp_per_capita', 'hospital_beds_per_thousand', 'life_expectancy', 'human_development_index', 'cases_growth_rate', 'death_rate', 'healthcare_capacity_index', 'date_vacc', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'vaccination_speed', 'vaccination_efficiency']


In [19]:
# Dataset para Regresión (predecir días a 70% vacunación)
regression_data = integrated_df[[
    'country', 'gdp_per_capita', 'hospital_beds_per_thousand',
    'population_density', 'median_age', 'healthcare_capacity_index',
    'people_fully_vaccinated_per_hundred', 'vaccination_speed'
]].copy()

# Dataset para Clasificación (nivel de riesgo)
classification_data = integrated_df[[
    'country', 'total_deaths_per_million', 'gdp_per_capita',
    'hospital_beds_per_thousand', 'human_development_index',
    'death_rate', 'healthcare_capacity_index'
]].copy()

print("\nDATASETS PARA MODELADO:")
print(f"\nRegresión: {regression_data.shape}")
print(f"Clasificación: {classification_data.shape}")

# Eliminar filas con missing
regression_data = regression_data.dropna()
classification_data = classification_data.dropna()

print(f"\nDespués de eliminar NA:")
print(f"Regresión: {regression_data.shape}")
print(f"Clasificación: {classification_data.shape}")


DATASETS PARA MODELADO:

Regresión: (262, 8)
Clasificación: (262, 7)

Después de eliminar NA:
Regresión: (229, 8)
Clasificación: (0, 7)


In [20]:
# Guardar datasets procesados
print("\nGUARDANDO DATOS PROCESADOS...")

# Guardar en data/03_primary/
covid_features.to_parquet('../data/03_primary/covid_features.parquet', index=False)
vacc_features.to_parquet('../data/03_primary/vacc_features.parquet', index=False)
integrated_df.to_parquet('../data/03_primary/integrated_data.parquet', index=False)

# Guardar datasets para ML en data/04_feature/
regression_data.to_parquet('../data/04_feature/regression_data.parquet', index=False)
classification_data.to_parquet('../data/04_feature/classification_data.parquet', index=False)

print("Archivos guardados:")
print("  - data/03_primary/covid_features.parquet")
print("  - data/03_primary/vacc_features.parquet")
print("  - data/03_primary/integrated_data.parquet")
print("  - data/04_feature/regression_data.parquet")
print("  - data/04_feature/classification_data.parquet")

print("\n✅ PREPARACIÓN DE DATOS COMPLETADA")


GUARDANDO DATOS PROCESADOS...
Archivos guardados:
  - data/03_primary/covid_features.parquet
  - data/03_primary/vacc_features.parquet
  - data/03_primary/integrated_data.parquet
  - data/04_feature/regression_data.parquet
  - data/04_feature/classification_data.parquet

✅ PREPARACIÓN DE DATOS COMPLETADA
